In [16]:
# !pip install facenet-pytorch

In [50]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import albumentations as A
import os
from pathlib import Path

from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
from albumentations.pytorch import ToTensorV2

In [51]:
## -- 이미지 불러오기 테스트

img_path = '/opt/ml/input/data/train/images_faceCrop/000008_female_Asian_58/mask1.jpg'
new_path = Path(img_path)
img = Image.open(new_path)
print(np.array(img).shape)
transform = A.Compose([ToTensorV2(), A.Normalize(0.5, 0.5, 0.5)])

img = transform(image=np.array(img))['image']
plt.imshow(img)

(224, 224, 3)


AttributeError: 'Tensor' object has no attribute 'astype'

In [9]:
def faceCrop(img, faces, w_ratio, h_ratio, x, y):
    x1, y1, x2, y2 = faces[0][0]
    width = x2 - x1
    height = y2 - y1
    
    new_x = int(x1 - width * w_ratio / 2)
    new_y = int(y1 - height * h_ratio / 2)
    
    if(new_x < 0):
        w_ratio = x1 / width
        new_x = 0
    if(new_y < 0):
        h_ratio = y1 / height
        new_y = 0
    
    new_width = int(width * (1 + w_ratio))
    new_height = int(height * (1 + h_ratio))
    
    if(new_x + new_width > x):
        new_width = x - new_x
    if(new_y + new_height > y):
        new_height = y - new_y 
        
    transform = A.Compose([
                        A.Crop(new_x, new_y, new_x+new_width, new_y+new_height),
                        A.Resize(224, 224)
                        ])
    

    img = transform(image=np.array(img))['image']
    return img

detector = MTCNN()
face = detector.detect(img)
plt.imshow(faceCrop(img, face, 0.3, 0.4, 384, 512))
# plt.imshow(faceCrop(img, face, 0.3, 0.4, 224,224))

TypeError: 'NoneType' object is not subscriptable

In [10]:
csv_path = '/opt/ml/input/data/train/full_path_three_label.csv'
df = pd.read_csv(csv_path)

In [12]:
train_dir = True
target_dir = 'train' if train_dir else 'eval'
seg_crop_img_dir_path = '/opt/ml/input/data/{}/images_seg_crop2'.format(target_dir)
orig_img_dir_path = '/opt/ml/input/data/{}/images_resize'.format(target_dir)

if not os.path.isdir(seg_crop_img_dir_path):
    os.mkdir(seg_crop_img_dir_path)

In [15]:
li = df["mask"].to_list()
normal_idx = []
for i in range(len(li)):
    if li[i] == 2:
        normal_idx.append(i)

In [29]:
count = 0
people_n = int(len(df)/7)
 

for idx in range(1, people_n):
    ori_normal_path = Path(df['path'].iloc[normal_idx[idx]].replace('images', 'images_resize'))
    ori_normal_img = Image.open(ori_normal_path)
    ori_normal_face = detector.detect(ori_normal_img)
    
    for j in range(7):
        ori_path = Path(df['path'].iloc[idx*7+j].replace('images', 'images_resize'))
        seg_path = Path(str(ori_path).replace('images_resize', 'images_segmented'))
        
        ori_img = Image.open(ori_path)
        seg_img = Image.open(seg_path)
        
        ori_face = detector.detect(ori_img)
        
        if(ori_face[1][0] != None):
            seg_crop_img = Image.fromarray(faceCrop(seg_img, ori_face, 0.3, 0.4, 224, 224))
        else:
            crop_img = Image.fromarray(faceCrop(seg_img, ori_normal_face, 0.3, 0.4, 224, 224))
            
            
        seg_crop_new_dir = Path(seg_crop_img_dir_path) / ori_path.parent.name
        if not os.path.isdir(seg_crop_new_dir):
            os.makedirs(seg_crop_new_dir)
            
        seg_crop_new_path = seg_crop_new_dir / ori_path.name
        seg_crop_img.save(seg_crop_new_path)
        
    if(idx % 100 == 0):
        print(idx)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
